This notebook provides an example of how the `selective` argument in the `create_data_matrix_for_pc` function can be used to restrict the study to a specific time frame or season of interest.

In [1]:
import numpy as np
import pandas as pd
from scipy.io import loadmat

Here we use sample data stored in the *sample_data.mat* matrix. The variables in this matrix `var1` and `var2` have already been detrended. The `time` variable contains the Year, Month and Day of each time sample.

In [2]:
# load sample data
input_mat = loadmat('sample_data.mat')
var1 = input_mat['var1']
var2 = input_mat['var2']
time = input_mat['time']

In [3]:
time_df = pd.DataFrame(time, columns = ['Year', 'Month', 'Day'])
time_df.head(10) # view the first 10 entries

,Year,Month,Day
0,402,1,2
1,402,1,7
2,402,1,12
3,402,1,17
4,402,1,22
5,402,1,27
6,402,2,1
7,402,2,6
8,402,2,11
9,402,2,16


In [4]:
data = np.hstack((var1.T, var2.T)) # N_samples x N_vars ndarray
[num_total_samples, num_vars] = np.shape(data)

In this example, we restrict our study to the winter season (Dec, Jan, Feb) by setting the `months_of_season = [12,1,2]`. 

In [5]:
# user defined parameters
months_of_season = [12,1,2] # the months to be used for the study
num_plus_minus_shifts = 3 # number of shifts in the positive or negative directions
num_shifts = 2*num_plus_minus_shifts # total number of time shifted variables
shift_dist = 1 # shift distance (D)

num_tiers = num_shifts+1 # number of tiers
num_nodes = num_tiers * num_vars # number of nodes in the graphical model

In [6]:
def create_data_matrix_for_pc(standardize = True, **kwargs):

    """
    Creates the input matrix for the PC stable algorithm. This function
    (1) Creates time shifted versions of the variables. 
    (2) The user can optionally specify the indices that should be used in the study
        by passing 'selective' and 'select_idx'. 
    (2) Standardizes each variable based on user preference.     
    """
    
    selective = kwargs.get('selective', None)
    select_idx = kwargs.get('select_idx', None)
     
    def create_time_shifted_matrix():
        
        "Creates a matrix containing time shifted versions of the variables as in Figure 1."
        
        num_samples_time_shifted_matrix = num_total_samples-num_shifts*shift_dist
        time_shifted_matrix = np.empty((num_samples_time_shifted_matrix, num_nodes)) # initialize
        for i in range(num_tiers):
            start_time_for_tier = shift_dist*i
            end_time_for_tier = num_total_samples - shift_dist*(num_shifts-i)
            current_cols = range(i*num_vars,(i+1)*num_vars)
            time_shifted_matrix[:, current_cols] = data[start_time_for_tier : end_time_for_tier,:] 

        return time_shifted_matrix
    
    def standardize_data(data):
        
        "Standardizes the data of each node"
        
        means = data.mean(axis=0)
        stds = data.std(axis=0)
    
        return (data - means) / stds 
       

    time_shifted_matrix = create_time_shifted_matrix()
   
    if(selective):
        select_matrix  = time_shifted_matrix[select_idx]
    else:
        select_matrix  = time_shifted_matrix

    if(standardize):
        pc_data = standardize_data(select_matrix) 
        
    else:
        pc_data = select_matrix
        
    return pc_data

The time window we use for this study is centered around Dec, Jan, and Feb, resulting in some samples from Nov and Mar also included in the analysis.

We can set the `selective` argument to **True** and provide `select_idx` to the `create_data_matrix_for_pc` function to ensure that only specific rows of the `time_shifted_matrix` are used for the study.

In [7]:
# idx = indices corresponding to the middle of the window (i.e., time 't')
idx = range(shift_dist*num_plus_minus_shifts , num_total_samples-shift_dist*num_plus_minus_shifts)
months = time[:,1][idx] # month of the indices in idx
mask = np.in1d(months, months_of_season) # check whether the month belongs to the months_of_season
season_idx = np.where(mask) # the row indices of time_shifted_matrix that should be used for the study

In [8]:
pc_data = create_data_matrix_for_pc(standardize = True, selective = True, select_idx = season_idx)